In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import time
import random
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [2]:
import tls_client
requests = tls_client.Session(
    client_identifier="chrome112",
)

In [3]:
months = ['october','november','december','january','february','march','april']

In [4]:
def get_month(month):
    url = f'https://www.basketball-reference.com/leagues/NBA_2024_games-{month}.html'
    response1 = requests.get(url)
    soup = BeautifulSoup(response1.content)
    rows = soup.findAll('th')
    dates = [[td.getText() for td in rows[i]]
                for i in range(11,(len(rows)))]

    for i in range(len(dates)):
        dates[i] = dates[i][0]
    i = 0
    while i < 40:
        headers = [th.getText() for th in soup.findAll('tr', limit=40)[i].findAll('th')]
        i = i +1
        if headers:
            idx = i
            i = 41
    headers = headers[1:]
    rows = soup.findAll('tr')
    player_stats = [[td.getText() for td in rows[i].findAll('td')]
                for i in range(1,len(rows))]
    stats = pd.DataFrame(player_stats, columns = headers)
    stats.insert(0, "Date", dates)
    return stats


for month in months:
    month_schedule = get_month(month)
    if month == 'october':
        schedule = month_schedule
    else:
        schedule = pd.concat((schedule,month_schedule))
schedule

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS,,,Attend.,Arena,Notes
0,"Tue, Oct 24, 2023",7:30p,Los Angeles Lakers,,Denver Nuggets,,,,,Ball Arena,
1,"Tue, Oct 24, 2023",10:00p,Phoenix Suns,,Golden State Warriors,,,,,Chase Center,
2,"Wed, Oct 25, 2023",7:00p,Houston Rockets,,Orlando Magic,,,,,Amway Center,
3,"Wed, Oct 25, 2023",7:00p,Boston Celtics,,New York Knicks,,,,,Madison Square Garden (IV),
4,"Wed, Oct 25, 2023",7:00p,Washington Wizards,,Indiana Pacers,,,,,Gainbridge Fieldhouse,
...,...,...,...,...,...,...,...,...,...,...,...
109,"Sun, Apr 14, 2024",3:30p,Phoenix Suns,,Minnesota Timberwolves,,,,,Target Center,
110,"Sun, Apr 14, 2024",3:30p,Los Angeles Lakers,,New Orleans Pelicans,,,,,Smoothie King Center,
111,"Sun, Apr 14, 2024",3:30p,Dallas Mavericks,,Oklahoma City Thunder,,,,,Paycom Center,
112,"Sun, Apr 14, 2024",3:30p,Portland Trail Blazers,,Sacramento Kings,,,,,Golden 1 Center,


In [5]:
schedule = schedule[['Date','Visitor/Neutral','Home/Neutral']]

In [6]:
acro = pd.read_csv('acronyms',index_col=0)

acro.columns

a = acro.iloc[:,0]
a = a.to_list()
b = acro.iloc[:,1]
b = b.to_list()

d = {b[i]:a[i] for i in range(len(a))}
d['Los Angeles Clippers']='LAC'
d

{'Atlanta Hawks': 'ATL',
 'Boston Celtics': 'BOS',
 'Charlotte Hornets': 'CHA',
 'Chicago Bulls': 'CHI',
 'Cleveland Cavaliers': 'CLE',
 'Dallas Mavericks': 'DAL',
 'Denver Nuggets': 'DEN',
 'Detroit Pistons': 'DET',
 'Golden State Warriors': 'GSW',
 'Houston Rockets': 'HOU',
 'Indiana Pacers': 'IND',
 'LA Clippers': 'LAC',
 'Los Angeles Lakers': 'LAL',
 'Memphis Grizzlies': 'MEM',
 'Miami Heat': 'MIA',
 'Milwaukee Bucks': 'MIL',
 'Minnesota Timberwolves': 'MIN',
 'New Orleans Pelicans': 'NOP',
 'New York Knicks': 'NYK',
 'Brooklyn Nets': 'BKN',
 'Oklahoma City Thunder': 'OKC',
 'Orlando Magic': 'ORL',
 'Philadelphia 76ers': 'PHI',
 'Phoenix Suns': 'PHX',
 'Portland Trail Blazers': 'POR',
 'Sacramento Kings': 'SAC',
 'San Antonio Spurs': 'SAS',
 'Toronto Raptors': 'TOR',
 'Utah Jazz': 'UTA',
 'Washington Wizards': 'WAS',
 'Los Angeles Clippers': 'LAC'}

In [8]:
for i in range(len(schedule)):
    if schedule.iloc[i,1] in d:
        schedule.iloc[i,1] = d[schedule.iloc[i,1]]
        
for i in range(len(schedule)):
    if schedule.iloc[i,2] in d:
        schedule.iloc[i,2] = d[schedule.iloc[i,2]]
schedule.head()

,Date,Visitor/Neutral,Home/Neutral
0,"Tue, Oct 24, 2023",LAL,DEN
1,"Tue, Oct 24, 2023",PHX,GSW
2,"Wed, Oct 25, 2023",HOU,ORL
3,"Wed, Oct 25, 2023",BOS,NYK
4,"Wed, Oct 25, 2023",WAS,IND


In [9]:
schedule['Home/Neutral'].unique()

array(['DEN', 'GSW', 'ORL', 'NYK', 'IND', 'CHA', 'MIA', 'TOR', 'BKN',
       'MEM', 'CHI', 'UTA', 'SAS', 'LAC', 'MIL', 'LAL', 'ATL', 'CLE',
       'BOS', 'DAL', 'POR', 'SAC', 'DET', 'WAS', 'NOP', 'MIN', 'PHX',
       'OKC', 'HOU', 'PHI'], dtype=object)

In [10]:
schedule['Date'] = pd.to_datetime(schedule['Date'])

C:\Users\Shared.Westover\AppData\Local\Temp\ipykernel_20836\2966185454.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schedule['Date'] = pd.to_datetime(schedule['Date'])


In [11]:
schedule.iloc[0,0]

Timestamp('2023-10-24 00:00:00')

In [12]:
schedule.to_csv('schedule.csv')